In [ ]:
!pip install python-resize-image
!pip install kaggle-cli

In [ ]:
!pip install --no-cache-dir -I pillow
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

https://forums.fast.ai/t/attributeerror-module-pil-image-has-no-attribute-register-extensions/10689/5

In [ ]:
import os
import re
import urllib
import urllib.request
import urllib.parse
from PIL import Image
from resizeimage import resizeimage

In [ ]:
# !pip install Pillow
import PIL
from PIL import Image

In [ ]:
!kg download -u login -p mdp -c dogs-vs-cats

In [ ]:
!mkdir dogsCats;cd dogCats;ls

In [ ]:
!mv test1.zip train.zip dogsCats/

In [ ]:
!cd dogsCats; unzip train.zip

In [ ]:
!cd dogsCats;rm -rf test1.zip train.zip;ls 

In [ ]:
def resize_img(imgpath, target_size = 150):
    try:
#         print (imgpath)
        img = Image.open(imgpath)
#         print (img.size)
        new_img = img.resize((target_size,target_size))
        new_img.save(imgpath, "JPEG", optimize=True)
    except:
#         print (imgpath)
        error.append(imgpath)

In [ ]:
# error = []
# imgLst = os.listdir('dogsCats/train')
# print(imgLst[0])
# resize_img('dogsCats/train/' + imgLst[0])
# print (error)

In [ ]:
!ls dogsCats

In [ ]:
error = []
imgs = []
path = 'dogsCats/'
for path, subdirs, files in os.walk(path):
    for name in files:
        imgs.append(os.path.join(path, name))

In [ ]:
print (imgs[1])

In [ ]:
for ind, img in enumerate(imgs):
    resize_img(img)
    if ind % 1000 == 0:
      print (ind)

In [ ]:
!find dogsCats/train -type f | wc -l

In [ ]:
print (len(error))

In [ ]:
!ls dogsCats

In [ ]:
for err in error:
  os.remove(err)

In [ ]:
from skimage.io import imread, imsave
from skimage.transform import resize

In [ ]:
error = []
for img in imgs:
    image = imread(img)
    res = resize(image, (224, 224), preserve_range=True, mode='reflect')
    if res.shape != (224, 224, 3):
        print (img)
        print (res.shape)
        error.append(img)

In [ ]:
print (len(error), error)
for err in error:
  os.remove(err)

In [ ]:
import tensorflow as tf
import keras as K

In [ ]:
print (tf.__version__, K.__version__)

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
import cv2 as cv
%pylab inline

import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg

import tensorflow as tf
import pandas as pd
import numpy as np

import seaborn as sns
%matplotlib inline

from keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU
# import keras_utils

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
BATCH_SIZE = 64
INIT_LR = 5e-3 

In [ ]:
!cd dogsCats/ ; ls

In [ ]:
path = './dogsCats/train'
imgs = []

In [ ]:
for path, subdirs, files in os.walk(path):
    for name in files:
        imgs.append(os.path.join(path, name))

In [ ]:
print (imgs[:5])

In [ ]:
lbls = []
for img in imgs:
    if 'cat' in img:
        lbls.append(0)
    else:
        lbls.append(1)

In [ ]:
print (len(imgs), len(lbls))

In [ ]:
lbls = keras.utils.to_categorical(lbls, num_classes=2)

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(imgs,
                                                      lbls,
                                                      test_size=0.2,
                                                      random_state=42)

In [ ]:
for ind in range(0,5):
    print (X_train[ind], y_train[ind])

In [ ]:
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])

In [ ]:
import numpy as np
from skimage.io import imread, imsave
from skimage.transform import resize

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(150,150), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def read_and_resize(self, filepath):
        img = imread(filepath)
#         img = preprocess_input(img)
        res = resize(img, self.dim, preserve_range=True, mode='reflect')
        return np.expand_dims(res, 0)
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = [self.read_and_resize(self.list_IDs[i])
             for i in indexes]
        y = self.labels[indexes]
        X = np.vstack(X)
        return seq.augment_images(X), y

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

def resnet():
    base_model = ResNet50(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    pred = Dense(2, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=pred)
    return model

In [ ]:
# from keras.metrics import top_k_categorical_accuracy
# with tf.device("/cpu:0"):
#     model = Model(inputs=base_model.input, outputs=pred)
K.clear_session()
model = resnet()

In [ ]:
model.compile(optimizer = 'Adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:


tensorboard = keras.callbacks.TensorBoard(log_dir='./logs/resnet2', 
                                          histogram_freq=0, 
                                          batch_size=BATCH_SIZE, 
                                          write_graph=True, 
                                          write_grads=True, 
                                          write_images=True, 
                                          embeddings_freq=0,
                                          embeddings_layer_names=None, 
                                          embeddings_metadata=None) 

In [ ]:
# train_gen = DataGenerator(X_train, y_train, batch_size = 32)
# val_gen = DataGenerator(X_test, y_test, batch_size = 32)
checkpoint = ModelCheckpoint('checkpoint.h5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.001)
earlyStopping = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')

In [ ]:
params = {
            'dim': (224,224),
            'batch_size': 32,
            'n_classes': 2,
            'n_channels': 3,
            'shuffle': True
         }

train_gen = DataGenerator(X_train, y_train, **params)
val_gen = DataGenerator(X_test, y_test, **params)

In [ ]:
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback pour afficher la learning rate
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

In [ ]:
for layer in model.layers:
    layer.trainable = True
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.momentum = 0.8
    
for layer in model.layers[:-20]:
    layer.trainable = False

In [ ]:
history = model.fit_generator(generator = train_gen,
                          validation_data = val_gen,
                          use_multiprocessing = True,
#                           callbacks=[tensorboard, keras.callbacks.LearningRateScheduler(lr_scheduler),
#                                      LrHistory(), earlyStopping],
                          callbacks=[tensorboard],
                          workers=3,
                          epochs = 1)

In [ ]:
# historique pour l'accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# historique pour la loss function
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

<b>Objectif:</b>
- atteindre les 90% sans overfitting